In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_057.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_055', 'Input_C_135', 'Input_C_052', 'Input_C_054',
       'Input_C_056', 'Input_C_059', 'Input_C_053'], dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.004042087916713229
test_rmse: 0.0042089979115283935
test_r2: 0.6829602163837639


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-05)

test_rmse_ave: 0.0041904108831711685
[0.004287509088716964, 0.003776694510842413, 0.00480016563914731, 0.004305953432606997, 0.0038289321858901337, 0.0041579762023212785, 0.0044878348324749115, 0.004106011221814918, 0.004378551486645267, 0.003774480231251494]


test_r2_ave: 0.6815897086733098
[0.6249772897576982, 0.7131852174572721, 0.5839568773674868, 0.685480870668556, 0.7190626184048907, 0.7048772824351657, 0.6253944941082511, 0.7168611221161302, 0.6931604028396188, 0.7489409115780302]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.006],0.01)

test_rmse_ave: 0.004193702972113819
[0.004286462440973182, 0.003775651289664121, 0.004802520122243878, 0.0043315897036147275, 0.0038299855778753163, 0.004159692826994745, 0.004486420957920593, 0.0041066982476830015, 0.004378165143431671, 0.00377984341073695]


test_r2_ave: 0.6811157193753848
[0.6251603651778102, 0.7133436469672969, 0.5835486386473212, 0.6817246293079218, 0.7189080176546351, 0.7046335487088236, 0.6256304928832274, 0.7167663634820671, 0.6932145486759472, 0.7482269422487982]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.004128782634767667
test_rmse: 0.00438183520690091
test_r2: 0.6562100947605988


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 150, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0044244270101083645
[0.0047573713880458245, 0.0038851971989134707, 0.0051997965474765715, 0.004417762197095977, 0.004260173402800977, 0.0038674117520307383, 0.0047338779087662914, 0.004131262650254948, 0.004935741071267833, 0.004055675984431014]


test_r2_ave: 0.6429179019156523
[0.5382769313827493, 0.6964683721230185, 0.5117990700613742, 0.6689351474159433, 0.6522166917063368, 0.7446831557877693, 0.5831934530888886, 0.7133678799093603, 0.6100982187246534, 0.7101400989564284]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 7,
            min_samples_split = 6,
            max_leaf_nodes = 14,
            min_samples_leaf =1,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.004210428000843861
[0.00450619574494823, 0.0036921869255320053, 0.004845711920871969, 0.004237000178915273, 0.0038846800422618305, 0.003916908995091113, 0.004537172274781583, 0.0040239100765399214, 0.0045645390100714145, 0.0038959748394252766]


test_r2_ave: 0.6776289684220653
[0.5857451831114955, 0.7258771885721371, 0.5760241857269011, 0.6954733030797475, 0.7108223728018738, 0.7381059659833445, 0.6171126965494411, 0.7280708430104056, 0.6665395527904143, 0.7325183925948928]


In [11]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.006], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.25, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'dart', 
              n_estimators = 150, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.7,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 7,
            min_samples_split = 6,
            max_leaf_nodes = 14,
            min_samples_leaf =1,
            random_state = 42)  
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('RandomForest', random_mod),('XGB', xgb_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.004169700256232622
[0.004293656572711867, 0.00371599159686002, 0.004855900319464273, 0.004269397102309296, 0.0038112024307395214, 0.004009383520080562, 0.004472433959170527, 0.004023749004322595, 0.004448175700075298, 0.003797112356592262]


test_r2_ave: 0.684382184857532
[0.6239010943025324, 0.7223310847540501, 0.5742394425128092, 0.6907985590219285, 0.7216583389219013, 0.725593847524356, 0.62796114506506, 0.7280926125603229, 0.6833245847551833, 0.7459211391571746]


In [17]:

stack_mod = StackingRegressor(regressors=[lasso_mod, lr, elastic_mod, random_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00418014312187139
[0.004336927764381224, 0.0037068340067700316, 0.004843298694177453, 0.004279981304114157, 0.003834978304245382, 0.003953260964824748, 0.004550209798610479, 0.0040241041453828235, 0.004452152316688206, 0.003819683919519392]


test_r2_ave: 0.6825690431448248
[0.6162822952841835, 0.7236979580976792, 0.5764463713618235, 0.6892635848971039, 0.718174683253032, 0.7332222465960714, 0.6149090886046407, 0.7280446126779995, 0.6827581234909184, 0.7428914671847964]


In [18]:
ave(skew_data[title],y,0.15,stack_mod,0.75,vote_mod,0.1,lasso_mod)

test_rmse_ave: 0.004158794255211677
[0.004284914734691216, 0.0037066006132283518, 0.004838589847067231, 0.004257468922255594, 0.0038003251894357572, 0.003995321926777176, 0.004473561246879752, 0.004018708885694085, 0.004431548335050916, 0.0037809028510366856]


test_r2_ave: 0.6860037671758704
[0.6254310018298801, 0.7237327506359277, 0.5772695622756032, 0.6925238884608256, 0.7232448560747713, 0.7275152508191248, 0.6277735748573888, 0.7287733643578062, 0.6856876373188217, 0.7480857851285545]
